In [7]:
# Standard modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sunpy.net import Fido, attrs

# Custom modules

__author__ = 'James Paul Mason'
__contact__ = 'jmason86@gmail.com'


def get_sdo_eve_lines_data(start_time, end_time, 
                           verbose=False):
    """Get emission lines data product from the Solar Dynamics Observatory / Extreme Ultraviolet Variability Experiment.

    Inputs:
        start_time [metatime]: The beginning of the time window of interest. See jpm_time_conversions.py
                               (https://github.com/jmason86/python_convenience_functions/blob/master/jpm_time_conversions.py)
                               for allowed metatime formats.
        end_time [metatime]:   Same as start time but for the end of the time window.
        
    Optional Inputs:
        verbose [bool]:                    Set to log the processing messages to disk and console. Default is False.

    Outputs:
        TBD
                                                 
    Optional Outputs:
        None

    Example:
        TBD
    """
    return 1

In [8]:
# prototype only: setup for test case
start_time = "2010-09-05 00:00:00"
end_time = "2010-09-05 01:00:00"

In [9]:
from astropy import units as u

In [12]:
results = Fido.search(attrs.Time("2010-09-05 00:00:00", "2010-09-05 01:00:00"), attrs.Instrument('eve'))

In [11]:
results = Fido.search(attrs.Time(start_time, end_time), attrs.Instrument('eve'), attrs.Wavelength(wavemin=62 * u.angstrom, wavemax=1035 * u.angstrom))
results

Start Time [1],End Time [1],Source,Instrument,Type,Wavelength [2]
,,,,,Angstrom
str19,str19,str3,str3,str8,float64
2010-09-05 00:00:00,2010-09-05 01:00:00,SDO,EVE,FULLDISK,60.0 .. 1060.0
2010-09-05 01:00:00,2010-09-05 02:00:00,SDO,EVE,FULLDISK,60.0 .. 1060.0
2010-09-05 00:00:00,2010-09-06 00:00:00,SDO,EVE,FULLDISK,1.0 .. 1050.0


In [13]:
# Suggestions from Stuart 
vr = results.get_response(0)
vr[2]['info']

L2Lines (merged)

In [39]:
# another suggestion from Stuart
bla = list(results.responses)
bla[0][1]['fileid']

EVE_L2_lines_2010248_00

In [46]:
np.shape(results)

(1, 6, 9, 2)

In [ ]:
files = Fido.fetch(results, path='./{instrument}/{fileid}/{info}/{file}.fits')
files

In [57]:
results.response_block_properties()

{'extent',
 'fileid',
 'info',
 'instrument',
 'physobs',
 'provider',
 'source',
 'time',
 'wave'}

In [1]:
from sunpy.instr.goes import get_goes_event_list
from sunpy.time import TimeRange

In [2]:
time_range = TimeRange('2010-05-01 00:00:00', '2018-01-11 00:00:00')

In [3]:
bla = get_goes_event_list(time_range, goes_class_filter='c1')

In [5]:
len(bla)

8537

In [ ]:
return tbd